# Zipline Backtest with Single Factor 

This notebook develops and test a simple mean-reversion factor that measures how much recent performance has deviated from the historical average. Short-term reversal is a common strategy that takes advantage of the weakly predictive pattern that stock price increases are likely to mean-revert back down over horizons from less than a minute to one month.

> Please see the installation [instructions](../installation/README.md) for setting up Zipline and fixing **known issues**. Use with `conda` environment `ml4t-zipline`

## Imports & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
sns.set_style('whitegrid')

We are first going to illustrate the zipline alpha factor research workflow in an offline environment. In particular, we will develop and test a simple mean-reversion factor that measures how much recent performance has deviated from the historical average. 

Short-term reversal is a common strategy that takes advantage of the weakly predictive pattern that stock price increases are likely to mean-revert back down over horizons from less than a minute to one month.

To this end, the factor computes the z-score for the last monthly return relative to the rolling monthly returns over the last year. At this point, we will not place any orders to simply illustrate the implementation of a CustomFactor and record the results during the simulation.

After some basic settings, `MeanReversion` subclasses `CustomFactor` and defines a `compute()` method. It creates default inputs of monthly returns over an also default year-long window so that the monthly_return variable will have 252 rows and one column for each security in the Quandl dataset on a given day.

The `compute_factors()` method creates a `MeanReversion` factor instance and creates long, short, and ranking pipeline columns. The former two contain Boolean values that could be used to place orders, and the latter reflects that overall ranking to evaluate the overall factor performance. Furthermore, it uses the built-in `AverageDollarVolume` factor to limit the computation to more liquid stocks

The result would allow us to place long and short orders. We will see in the next chapter how to build a portfolio by choosing a rebalancing period and adjusting portfolio holdings as new signals arrive.

- The `initialize()` method registers the compute_factors() pipeline, and the before_trading_start() method ensures the pipeline runs on a daily basis. 
- The `record()` function adds the pipeline's ranking column as well as the current asset prices to the performance DataFrame returned by the `run_algorithm()` function

We will use the factor and pricing data stored in the performance DataFrame to evaluate the factor performance for various holding periods in the next section, but first, we'll take a look at how to create more complex signals by combining several alpha factors from a diverse set of data sources on the Quantopian platform.

Run using jupyter notebook extension

In [4]:
%load_ext zipline

In [5]:
%%zipline --start 2015-1-1 --end 2018-1-1 --output single_factor.pickle

from zipline.api import (
    attach_pipeline,
    date_rules,
    time_rules,
    order_target_percent,
    pipeline_output,
    record,
    schedule_function,
    get_open_orders,
    calendars
)
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume
import numpy as np
import pandas as pd

MONTH = 21
YEAR = 12 * MONTH
N_LONGS = N_SHORTS = 25
VOL_SCREEN = 1000


class MeanReversion(CustomFactor):
    """Compute ratio of latest monthly return to 12m average,
       normalized by std dev of monthly returns"""
    inputs = [Returns(window_length=MONTH)]
    window_length = YEAR

    def compute(self, today, assets, out, monthly_returns):
        df = pd.DataFrame(monthly_returns)
        out[:] = df.iloc[-1].sub(df.mean()).div(df.std())


def compute_factors():
    """Create factor pipeline incl. mean reversion,
        filtered by 30d Dollar Volume; capture factor ranks"""
    mean_reversion = MeanReversion()
    dollar_volume = AverageDollarVolume(window_length=30)
    return Pipeline(columns={'longs': mean_reversion.bottom(N_LONGS),
                             'shorts': mean_reversion.top(N_SHORTS),
                             'ranking': mean_reversion.rank(ascending=False)},
                    screen=dollar_volume.top(VOL_SCREEN))


def exec_trades(data, assets, target_percent):
    """Place orders for assets using target portfolio percentage"""
    for asset in assets:
        if data.can_trade(asset) and not get_open_orders(asset):
            order_target_percent(asset, target_percent)


def rebalance(context, data):
    """Compute long, short and obsolete holdings; place trade orders"""
    factor_data = context.factor_data
    record(factor_data=factor_data.ranking)

    assets = factor_data.index
    record(prices=data.current(assets, 'price'))

    longs = assets[factor_data.longs]
    shorts = assets[factor_data.shorts]
    divest = set(context.portfolio.positions.keys()) - set(longs.union(shorts))

    exec_trades(data, assets=divest, target_percent=0)
    exec_trades(data, assets=longs, target_percent=1 / N_LONGS)
    exec_trades(data, assets=shorts, target_percent=-1 / N_SHORTS)


def initialize(context):
    """Setup: register pipeline, schedule rebalancing,
        and set trading params"""
    attach_pipeline(compute_factors(), 'factor_pipeline')
    schedule_function(rebalance,
                      date_rules.week_start(),
                      time_rules.market_open(),
                      calendar=calendars.US_EQUITIES)
    context.set_commission(commission.PerShare(cost=.01, min_trade_cost=0))
    context.set_slippage(slippage.VolumeShareSlippage())


def before_trading_start(context, data):
    """Run factor pipeline"""
    context.factor_data = pipeline_output('factor_pipeline')

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2015-01-02 21:00:00+00:00,NaN,0.000000,NaN,0.002012,NaN,NaN,0.000000e+00,1.000000e+07,0.000,0.000,...,0.000,0.000,0,NaN,1.000000e+07,0.000,0.000,1,[],0.0
2015-01-05 21:00:00+00:00,0.000000,0.000000,NaN,0.004029,0.000000,NaN,0.000000e+00,1.000000e+07,0.000,0.000,...,0.000,0.000,0,NaN,1.000000e+07,0.000,0.000,2,[],0.0
2015-01-06 21:00:00+00:00,0.003482,-0.000380,NaN,0.006049,0.000000,NaN,-3.118062e+06,6.881938e+06,3114262.860,3114262.860,...,-1617262.705,-1617262.705,4,-9.165151,1.000000e+07,0.000,0.000,3,[{'order_id': '9e34867f8bd64905ac33cded25b4e63...,0.0
2015-01-07 21:00:00+00:00,0.011564,0.000905,NaN,0.008073,0.000000,NaN,0.000000e+00,6.881938e+06,3127113.440,3127113.440,...,-1629987.410,-1629987.410,4,18.918320,6.881938e+06,3114262.860,3114262.860,4,[],0.0
2015-01-08 21:00:00+00:00,0.044725,0.007277,NaN,0.010102,0.000000,NaN,0.000000e+00,6.881938e+06,3190835.200,3190835.200,...,-1645106.080,-1645106.080,4,135.877107,6.881938e+06,3127113.440,3127113.440,5,[],0.0
2015-01-09 21:00:00+00:00,0.052254,0.003517,NaN,0.012134,0.000000,NaN,0.000000e+00,6.881938e+06,3153233.140,3153233.140,...,-1671113.400,-1671113.400,4,6.112254,6.881938e+06,3190835.200,3190835.200,6,[],0.0
2015-01-12 21:00:00+00:00,0.049097,0.002165,NaN,0.014171,0.000000,NaN,0.000000e+00,6.881938e+06,3139710.250,3139710.250,...,-1659971.660,-1659971.660,4,3.297888,6.881938e+06,3153233.140,3153233.140,7,[],0.0
2015-01-13 21:00:00+00:00,0.046532,0.004256,NaN,0.016212,0.000000,NaN,1.662399e+06,8.544337e+06,1498219.590,1498219.590,...,-1615750.780,-1615750.780,4,6.021850,6.881938e+06,3139710.250,3139710.250,8,[{'order_id': '8dfbcd2cf23e477eb9e38296e8468fb...,0.0
2015-01-14 21:00:00+00:00,0.045621,0.002199,NaN,0.018257,0.000000,NaN,4.253524e+04,8.586872e+06,1435118.820,1435118.820,...,-1587033.870,-1587033.870,4,2.632596,8.544337e+06,1498219.590,1498219.590,9,[{'order_id': '24e56344600d4fefa5761f488145459...,0.0
2015-01-15 21:00:00+00:00,0.043015,0.002354,NaN,0.020306,0.000000,NaN,0.000000e+00,8.586872e+06,1436672.200,1436672.200,...,-1561330.410,-1561330.410,4,2.671096,8.586872e+06,1435118.820,1435118.820,10,[],0.0
